# Heuristic learning

In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers, models, Input
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping


# 1200 dataset attempt cnn

In [35]:
dataset = np.load("star_battle_dataset.npz", allow_pickle=True)
puzzles = dataset["puzzles"]
solutions = dataset["solutions"]
# Check the shape of the dataset
print("Puzzles shape:", puzzles.shape)
print("Solutions shape:", solutions.shape)

Puzzles shape: (1200, 10, 10)
Solutions shape: (1200, 10, 10)


## data pre processing

In [40]:
# augment data for larger dataset
def augment_data(puzzles, solutions):
    augmented_puzzles = []
    augmented_solutions = []
    
    for i in range(len(puzzles)):
        puzzle = puzzles[i]
        solution = solutions[i]
        
        # original puzzle
        augmented_puzzles.append(puzzle)
        augmented_solutions.append(solution)
        
        # horizontal Flip
        augmented_puzzles.append(np.fliplr(puzzle))
        augmented_solutions.append(np.fliplr(solution))
        
        # vertical Flip
        augmented_puzzles.append(np.flipud(puzzle))
        augmented_solutions.append(np.flipud(solution))
        
        # rotate 90 degrees
        augmented_puzzles.append(np.rot90(puzzle, k=1))
        augmented_solutions.append(np.rot90(solution, k=1))
        
        # rotate 180 degrees
        augmented_puzzles.append(np.rot90(puzzle, k=2))
        augmented_solutions.append(np.rot90(solution, k=2))
        
        # rotate 270 degrees
        augmented_puzzles.append(np.rot90(puzzle, k=3))
        augmented_solutions.append(np.rot90(solution, k=3))
    
    return np.array(augmented_puzzles), np.array(augmented_solutions)

# apply augmentation to your dataset
augmented_puzzles, augmented_solutions = augment_data(puzzles, solutions)

print("Original puzzle count:", len(puzzles))
print("Augmented puzzle count:", len(augmented_puzzles))


Original puzzle count: 1200
Augmented puzzle count: 7200


In [42]:
# Reshape puzzles and solutions if necessary (ensure each puzzle is a 2D grid)
augmented_puzzles = puzzles[..., np.newaxis] # Assuming 10x10 grids, change if needed
augmented_solutions = solutions[..., np.newaxis]  # Same for solutions

# Normalize puzzle values between 0 and 1 (if necessary)
augmented_puzzles = puzzles.astype(np.float32) / 9.0

augmented_solutions = solutions.astype(np.float32) 

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(augmented_puzzles, augmented_solutions, test_size=0.2, random_state=42)

print(f"Training data shape: {X_train.shape}, {y_train.shape}")
print(f"Testing data shape: {X_test.shape}, {y_test.shape}")


Training data shape: (960, 10, 10, 1), (960, 10, 10, 1)
Testing data shape: (240, 10, 10, 1), (240, 10, 10, 1)


## build model

In [48]:
# Build the CNN model
model = models.Sequential([
    layers.Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(10, 10, 1)),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(128, (1, 1), activation='relu', padding='same'),
    layers.BatchNormalization(),

    layers.Flatten(),
    layers.Dense(100, activation='sigmoid'),  # Predicts for all 100 cells
    layers.Reshape((10, 10, 1))  # reshape back to 10x10 grid
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Summary of the model
model.summary()



Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_22 (Conv2D)          (None, 10, 10, 64)        640       
                                                                 
 batch_normalization (BatchN  (None, 10, 10, 64)       256       
 ormalization)                                                   
                                                                 
 max_pooling2d_21 (MaxPoolin  (None, 5, 5, 64)         0         
 g2D)                                                            
                                                                 
 conv2d_23 (Conv2D)          (None, 5, 5, 64)          36928     
                                                                 
 batch_normalization_1 (Batc  (None, 5, 5, 64)         256       
 hNormalization)                                                 
                                                      

## train model

In [49]:
# Train the model
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/20
30/30 [==============================] - 3s 41ms/step - loss: 0.6321 - accuracy: 0.7105 - val_loss: 0.6663 - val_accuracy: 0.6349
Epoch 2/20
30/30 [==============================] - 0s 11ms/step - loss: 0.5165 - accuracy: 0.7885 - val_loss: 0.6531 - val_accuracy: 0.6305
Epoch 3/20
30/30 [==============================] - 0s 11ms/step - loss: 0.4893 - accuracy: 0.7966 - val_loss: 0.6357 - val_accuracy: 0.6641
Epoch 4/20
30/30 [==============================] - 0s 12ms/step - loss: 0.4729 - accuracy: 0.8001 - val_loss: 0.6262 - val_accuracy: 0.6658
Epoch 5/20
30/30 [==============================] - 0s 12ms/step - loss: 0.4569 - accuracy: 0.8030 - val_loss: 0.6222 - val_accuracy: 0.6737
Epoch 6/20
30/30 [==============================] - 0s 12ms/step - loss: 0.4422 - accuracy: 0.8077 - val_loss: 0.6201 - val_accuracy: 0.6817
Epoch 7/20
30/30 [==============================] - 0s 11ms/step - loss: 0.4264 - accuracy: 0.8136 - val_loss: 0.6219 - val_accuracy: 0.6806
Epoch 8/20
30

## evaluate model

In [50]:
# Evaluate on test set
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_accuracy:.4f}")


8/8 [==============================] - 0s 4ms/step - loss: 0.6561 - accuracy: 0.7250
Test Accuracy: 0.7250


# 2500 dataset attempt

check if increase in data set significantly improves the model

In [11]:
dataset = np.load("star_battle_dataset.npz", allow_pickle=True)
puzzles = dataset["puzzles"]
solutions = dataset["solutions"]
# Check the shape of the dataset
print("Puzzles shape:", puzzles.shape)
print("Solutions shape:", solutions.shape)

Puzzles shape: (2500, 10, 10)
Solutions shape: (2500, 10, 10)


## preprocessing

In [14]:
# augment data for larger dataset
def augment_data(puzzles, solutions):
    augmented_puzzles = []
    augmented_solutions = []
    
    for i in range(len(puzzles)):
        puzzle = puzzles[i]
        solution = solutions[i]
        
        # original puzzle
        augmented_puzzles.append(puzzle)
        augmented_solutions.append(solution)
        
        # horizontal Flip
        augmented_puzzles.append(np.fliplr(puzzle))
        augmented_solutions.append(np.fliplr(solution))
        
        # vertical Flip
        augmented_puzzles.append(np.flipud(puzzle))
        augmented_solutions.append(np.flipud(solution))
        
        # rotate 90 degrees
        augmented_puzzles.append(np.rot90(puzzle, k=1))
        augmented_solutions.append(np.rot90(solution, k=1))
        
        # rotate 180 degrees
        augmented_puzzles.append(np.rot90(puzzle, k=2))
        augmented_solutions.append(np.rot90(solution, k=2))
        
        # rotate 270 degrees
        augmented_puzzles.append(np.rot90(puzzle, k=3))
        augmented_solutions.append(np.rot90(solution, k=3))
    
    return np.array(augmented_puzzles), np.array(augmented_solutions)

# apply augmentation to your dataset
augmented_puzzles, augmented_solutions = augment_data(puzzles, solutions)

print("Original puzzle count:", len(puzzles))
print("Augmented puzzle count:", len(augmented_puzzles))


Original puzzle count: 2500
Augmented puzzle count: 15000


In [15]:
# Reshape puzzles and solutions if necessary (ensure each puzzle is a 2D grid)
augmented_puzzles2500 = augmented_puzzles[..., np.newaxis] # Assuming 10x10 grids, change if needed
augmented_solutions2500 = augmented_solutions[..., np.newaxis]  # Same for solutions

# Normalize puzzle values between 0 and 1 (if necessary)
augmented_puzzles2500 = puzzles.astype(np.float32) / 9.0

augmented_solutions2500 = solutions.astype(np.float32) 

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(augmented_puzzles2500, augmented_solutions2500, test_size=0.2, random_state=42)

print(f"Training data shape: {X_train.shape}, {y_train.shape}")
print(f"Testing data shape: {X_test.shape}, {y_test.shape}")

Training data shape: (2000, 10, 10), (2000, 10, 10)
Testing data shape: (500, 10, 10), (500, 10, 10)


## replicate model

In [29]:
# Build the CNN model
model2 = models.Sequential([
    layers.Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(10, 10, 1)),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(128, (1, 1), activation='relu', padding='same'),
    layers.BatchNormalization(),

    layers.Flatten(),
    layers.Dense(100, activation='sigmoid'),  # Predicts for all 100 cells
    layers.Reshape((10, 10, 1))  # reshape back to 10x10 grid
])

# Compile the model
model2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Summary of the model
model2.summary()



Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_9 (Conv2D)               │ (None, 10, 10, 64)     │           640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_9           │ (None, 10, 10, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 5, 5, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 5, 5, 64)       │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_10          │ (None, 5, 5, 64)       │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 2, 2, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 2, 2, 128)      │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_11          │ (None, 2, 2, 128)      │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 100)            │        51,300 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape_3 (Reshape)             │ (None, 10, 10, 1)      │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 98,212 (383.64 KB)

 Trainable params: 97,700 (381.64 KB)

 Non-trainable params: 512 (2.00 KB)

## train model

In [30]:
history = model2.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/20


63/63 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.6940 - loss: 0.6424 - val_accuracy: 0.6529 - val_loss: 0.6498
Epoch 2/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7959 - loss: 0.5091 - val_accuracy: 0.6807 - val_loss: 0.6180
Epoch 3/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7986 - loss: 0.4941 - val_accuracy: 0.7088 - val_loss: 0.5977
Epoch 4/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7995 - loss: 0.4862 - val_accuracy: 0.7217 - val_loss: 0.5832
Epoch 5/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8005 - loss: 0.4762 - val_accuracy: 0.7367 - val_loss: 0.5701
Epoch 6/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8014 - loss: 0.4683 - val_accuracy: 0.7547 - val_loss: 0.5558
Epoch 7/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8041 - loss: 0.4585 - val_accuracy: 0.7781 - val_loss: 0.5390
Epoch 8/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8053 - loss: 0.4512 - val_accuracy: 0.7844 - val_loss: 0.5327
Epo

## evaluate model

In [31]:
# Evaluate on test set
test_loss, test_accuracy = model2.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_accuracy:.4f}")

16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7379 - loss: 0.6922 
Test Accuracy: 0.7377


# New approach

## u-net architecture with learning

## data preprocessing

In [2]:
dataset = np.load("star_battle_dataset.npz", allow_pickle=True)
puzzles = dataset["puzzles"]
solutions = dataset["solutions"]
# Check the shape of the dataset
print("Puzzles shape:", puzzles.shape)
print("Solutions shape:", solutions.shape)

Puzzles shape: (2500, 10, 10)
Solutions shape: (2500, 10, 10)


In [3]:
# augment data for larger dataset
def augment_data(puzzles, solutions):
    augmented_puzzles = []
    augmented_solutions = []
    
    for i in range(len(puzzles)):
        puzzle = puzzles[i]
        solution = solutions[i]
        
        # original puzzle
        augmented_puzzles.append(puzzle)
        augmented_solutions.append(solution)
        
        # horizontal Flip
        augmented_puzzles.append(np.fliplr(puzzle))
        augmented_solutions.append(np.fliplr(solution))
        
        # vertical Flip
        augmented_puzzles.append(np.flipud(puzzle))
        augmented_solutions.append(np.flipud(solution))
        
        # rotate 90 degrees
        augmented_puzzles.append(np.rot90(puzzle, k=1))
        augmented_solutions.append(np.rot90(solution, k=1))
        
        # rotate 180 degrees
        augmented_puzzles.append(np.rot90(puzzle, k=2))
        augmented_solutions.append(np.rot90(solution, k=2))
        
        # rotate 270 degrees
        augmented_puzzles.append(np.rot90(puzzle, k=3))
        augmented_solutions.append(np.rot90(solution, k=3))
    
    return np.array(augmented_puzzles), np.array(augmented_solutions)

# apply augmentation to your dataset
augmented_puzzles, augmented_solutions = augment_data(puzzles, solutions)

print("Original puzzle count:", len(puzzles))
print("Augmented puzzle count:", len(augmented_puzzles))


Original puzzle count: 2500
Augmented puzzle count: 15000


In [4]:
print(augmented_puzzles.shape)
print(augmented_solutions.shape)

(15000, 10, 10)
(15000, 10, 10)


In [5]:
# Reshape puzzles and solutions if necessary (ensure each puzzle is a 2D grid)
augmented_puzzles = augmented_puzzles[..., np.newaxis] # Assuming 10x10 grids, change if needed
augmented_solutions = augmented_solutions[..., np.newaxis]  # Same for solutions

# Normalize puzzle values between 0 and 1 (if necessary)
X = augmented_puzzles.astype(np.float32).reshape(-1, 10, 10, 1) / 9.0
y = augmented_solutions.astype(np.float32).reshape(-1, 10, 10, 1)

# split into training/validation, and testing datas
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.10, random_state=42, shuffle=True)

# split train and validation set from 90% of total data
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.1111, random_state=42, shuffle=True)

# Check splits
print(f"Train size: {len(X_train)}")
print(f"Val size: {len(X_val)}")
print(f"Test size: {len(X_test)}")

# print(X_train.shape)

Train size: 12000
Val size: 1500
Test size: 1500


## building unet architecture model

In [6]:
def build_unet(input_shape=(10, 10, 1), filters=64, learning_rate=0.001, dropout_rate=0.5, optimizer='adam'):
    inputs = layers.Input(shape=input_shape)

    # Encoder
    c1 = layers.Conv2D(filters, (3, 3), activation='relu', padding='same')(inputs)
    c2 = layers.Conv2D(filters * 2, (3, 3), activation='relu', padding='same')(c1)
    p1 = layers.MaxPooling2D((2, 2))(c2)

    # Decoder
    u1 = layers.Conv2DTranspose(filters, (3, 3), strides=(2, 2), padding='same')(p1)
    m1 = layers.Concatenate()([u1, c2])
    c3 = layers.Conv2D(filters, (3, 3), activation='relu', padding='same')(m1)
    
    # Apply Dropout
    c3 = layers.Dropout(dropout_rate)(c3)

    c4 = layers.Conv2D(1, (3, 3), activation='sigmoid', padding='same')(c3)

    # Choose optimizer
    if optimizer == 'adam':
        optimizer = Adam(learning_rate=learning_rate)
    else:
        optimizer = SGD(learning_rate=learning_rate)

    model = models.Model(inputs=inputs, outputs=c4)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model


In [34]:
model = build_unet()
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

## model training and hyperparameter tuning

In [8]:
early_stopping = EarlyStopping(patience=5, restore_best_weights=True)

In [37]:
# train model
history = model.fit(
    X_train, y_train,
    epochs=50,
    batch_size=32,
    validation_data=(X_val, y_val),
    callbacks=[early_stopping]
)

Epoch 1/50
422/422 ━━━━━━━━━━━━━━━━━━━━ 10s 23ms/step - accuracy: 0.8068 - loss: 0.4514 - val_accuracy: 0.8068 - val_loss: 0.4506
Epoch 2/50
422/422 ━━━━━━━━━━━━━━━━━━━━ 11s 27ms/step - accuracy: 0.8090 - loss: 0.4421 - val_accuracy: 0.8083 - val_loss: 0.4438
Epoch 3/50
422/422 ━━━━━━━━━━━━━━━━━━━━ 11s 27ms/step - accuracy: 0.8111 - loss: 0.4352 - val_accuracy: 0.8099 - val_loss: 0.4404
Epoch 4/50
422/422 ━━━━━━━━━━━━━━━━━━━━ 11s 27ms/step - accuracy: 0.8140 - loss: 0.4268 - val_accuracy: 0.8103 - val_loss: 0.4377
Epoch 5/50
422/422 ━━━━━━━━━━━━━━━━━━━━ 12s 28ms/step - accuracy: 0.8168 - loss: 0.4193 - val_accuracy: 0.8116 - val_loss: 0.4351
Epoch 6/50
422/422 ━━━━━━━━━━━━━━━━━━━━ 11s 27ms/step - accuracy: 0.8193 - loss: 0.4121 - val_accuracy: 0.8122 - val_loss: 0.4318
Epoch 7/50
422/422 ━━━━━━━━━━━━━━━━━━━━ 11s 27ms/step - accuracy: 0.8226 - loss: 0.4048 - val_accuracy: 0.8127 - val_loss: 0.4318
Epoch 8/50
422/422 ━━━━━━━━━━━━━━━━━━━━ 11s 27ms/step - accuracy: 0.8248 - loss: 0.3974 - 

In [38]:
from sklearn.metrics import classification_report

print(classification_report(y_val.flatten(), model.predict(X_val).flatten() > 0.5))


47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
              precision    recall  f1-score   support

         0.0       0.83      0.97      0.89    120000
         1.0       0.61      0.18      0.27     30000

    accuracy                           0.81    150000
   macro avg       0.72      0.57      0.58    150000
weighted avg       0.78      0.81      0.77    150000



## hyperparameter tuning

In [9]:
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split

# Model function with all hyperparameters included
def build_unet(input_shape=(10, 10, 1), filters=64, learning_rate=0.001, dropout_rate=0.5, optimizer='adam'):
    inputs = layers.Input(shape=input_shape)

    # Encoder
    c1 = layers.Conv2D(filters, (3, 3), activation='relu', padding='same')(inputs)
    c2 = layers.Conv2D(filters * 2, (3, 3), activation='relu', padding='same')(c1)
    p1 = layers.MaxPooling2D((2, 2))(c2)

    # Decoder
    u1 = layers.Conv2DTranspose(filters, (3, 3), strides=(2, 2), padding='same')(p1)
    m1 = layers.Concatenate()([u1, c2])
    c3 = layers.Conv2D(filters, (3, 3), activation='relu', padding='same')(m1)
    
    # Apply Dropout
    c3 = layers.Dropout(dropout_rate)(c3)

    c4 = layers.Conv2D(1, (3, 3), activation='sigmoid', padding='same')(c3)

    # Choose optimizer
    if optimizer == 'adam':
        optimizer = Adam(learning_rate=learning_rate)
    else:
        optimizer = SGD(learning_rate=learning_rate)

    model = models.Model(inputs=inputs, outputs=c4)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Manually loop over hyperparameters for grid search
param_grid = {
    'learning_rate': [0.001, 0.01, 0.0001],
    'filters': [32, 64, 128],
    'batch_size': [16, 32, 64],
    'epochs': [10, 20, 30],
    'dropout_rate': [0.3, 0.5],
    'optimizer': ['adam', 'sgd'],
}

# Assume X_train, y_train are your data and labels
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=42)

best_score = 0
best_params = {}

# Grid Search Loop
for lr in param_grid['learning_rate']:
    for f in param_grid['filters']:
        for bs in param_grid['batch_size']:
            for ep in param_grid['epochs']:
                for dr in param_grid['dropout_rate']:
                    for opt in param_grid['optimizer']:
                        print(f"Training with lr={lr}, filters={f}, batch_size={bs}, epochs={ep}, dropout={dr}, optimizer={opt}")
                        
                        model = build_unet(input_shape=(10, 10, 1), filters=f, learning_rate=lr, dropout_rate=dr, optimizer=opt)
                        model.fit(X_train, y_train, epochs=ep, batch_size=bs, validation_data=(X_val, y_val), callbacks=[early_stopping])
                        
                        # Evaluate the model on validation set
                        score = model.evaluate(X_val, y_val, verbose=0)
                        if score[1] > best_score:  # score[1] is accuracy
                            best_score = score[1]
                            best_params = {'learning_rate': lr, 'filters': f, 'batch_size': bs, 'epochs': ep, 'dropout_rate': dr, 'optimizer': opt}

print(f"Best parameters: {best_params} with accuracy: {best_score}")


Training with lr=0.001, filters=32, batch_size=16, epochs=10, dropout=0.3, optimizer=adam
Epoch 1/10
844/844 [==============================] - 12s 13ms/step - loss: 0.4971 - accuracy: 0.7993 - val_loss: 0.4829 - val_accuracy: 0.8005
Epoch 2/10
844/844 [==============================] - 10s 12ms/step - loss: 0.4789 - accuracy: 0.8008 - val_loss: 0.4735 - val_accuracy: 0.8016
Epoch 3/10
844/844 [==============================] - 8s 10ms/step - loss: 0.4713 - accuracy: 0.8016 - val_loss: 0.4673 - val_accuracy: 0.8025
Epoch 4/10
844/844 [==============================] - 8s 10ms/step - loss: 0.4654 - accuracy: 0.8027 - val_loss: 0.4621 - val_accuracy: 0.8035
Epoch 5/10
844/844 [==============================] - 9s 11ms/step - loss: 0.4602 - accuracy: 0.8039 - val_loss: 0.4587 - val_accuracy: 0.8040
Epoch 6/10
844/844 [==============================] - 9s 11ms/step - loss: 0.4558 - accuracy: 0.8050 - val_loss: 0.4570 - val_accuracy: 0.8045
Epoch 7/10
844/844 [==============================

KeyboardInterrupt: 

## gridcam visualisation??

In [ ]:
# gridcam visualisation